In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import chain
import operator

In [ ]:
df = pd.read_excel('drive/MyDrive/Prueba_tecnica/Datos entrevista.xlsx', skiprows=2)

# Preprocesamiento

In [ ]:
unnamed = df.loc[:, df.columns.str.startswith('Unnamed')].columns
df.drop(unnamed, axis = 1, inplace = True)
df = df.iloc[:48,:] # Se seleccionan unicamente las filas del documento que tienen los datos.

In [ ]:
# Separacion del documento en 4, para procesarlos independientemente
ventas = df.iloc[:,:-5]
pernoc = df.iloc[:,-5:-2]
pernoc.rename(columns = {'Fecha.1':'Fecha'}, inplace = True)
paro = df.iloc[:,-2:]

In [ ]:
# En excel, las ventas de cada mes aparecen como el dia uno. Lo pasamos al última día del mes para más claridad
ventas['Fecha'] = ventas['Fecha'] + pd.offsets.MonthEnd()

In [ ]:
# Igualamos la longitud de los datos de paro y datos de ventas, imputando los datos trimestrales a cada mes respectivo
valorparo = [[i]*3 for i in paro['Tasa de paro (INE) '].values]
paro = list(chain(*valorparo))[:48]

In [ ]:
parodf = pd.DataFrame()
parodf['paro'] = paro

## Web Scraping para obtener los cambios de precios de las principales marcas de tabaco en España

In [ ]:
import requests
import re
from datetime import datetime

In [ ]:
marcasinteres = ['Marlboro', 'Chesterfield', 'Camel', 'Winston', 'Fortuna', 'Nobel', 'Lucky Strike', 'Ducados', 'L&M', 'Pall Mall']
nombres_xlsx = [] # Guardamos esto para mas tarde poder iterar sobre los archivos guardados.
for marca in marcasinteres: # iteramos por cada marca de interes.
  url = "https://www.elpreciodeltabaco.com/"
  r = requests.get(url)
  comienzo = r.text.find('<li><div class=')
  final = r.text.find('No hay registros anteriores.')
  tablageneral = r.text[comienzo:final] # Aqui obtenemos la tabla de la pagina principal en HTML, la usaremos para guardar los identificadores de cada poducto de las marcas de interes. Son necesarios para poder acceder a la pagina
                                        # con la tabla de cambios de precios dentro de cada producto de la marca.
  lineas_marca = [ x for x in tablageneral.split('\n') if marca in x ]
  numeros_marca = [i.split('/')[2] for i in lineas_marca] # estos son los identificadores de cada producto. Ej: Marlboro_Red_Duro se identifica con 262 en la pagina web.
  nombres_marca = [i.split('/')[3].split('>')[0][:-1] for i in lineas_marca] # nombres del producto

  for idx,_ in enumerate(numeros_marca): # iteramos por producto dentro de la marca.
    numero_producto = numeros_marca[idx]
    producto = nombres_marca[idx]
    url = "https://www.elpreciodeltabaco.com/marca/{}/{}".format(numero_producto,producto) # definir el URL de forma "automatica" de cada producto con su nombre e identificador.
    r = requests.get(url)

    comienzo = r.text.find('<li><div class=')
    final = r.text.find('No hay registros anteriores.')
    tabla = r.text[comienzo:final] # definimos la parte del HTML que contiene la tabla.

    fechas = []
    pmaquina = []
    precio = []

    for i in tabla.split('\n')[1:]: # cada linea de la tabla (separadas por un salto de linea), representa un cambio de precio. Guardamos la fecha en la que ocurrió, el nuevo precio en maquinas y el precio en estancos.
      date_obj= i.split('>')[5][:-5]
      date_obj2 = datetime.strptime(date_obj, '%d/%m/%Y')
      new_date_str = date_obj2.strftime('%Y/%m/%d')
      fechas.append(new_date_str)
      pmaquina.append(float(i.split('>')[7][0:4].replace(",", ".")))
      precio.append(float(i.split('>')[9][0:4].replace(",", ".")))

    if (int(fechas[-1][:4])<=2013) and (int(fechas[-1][5:7])==1): # Solo guardo productos que aparecieron en enero 2013 o anteriormente, ya que los datos de ventas empiezan ahi.
      nombres_xlsx.append('drive/MyDrive/Prueba_tecnica/{}_{}.xlsx'.format(marca, producto))
      dfproducto = pd.DataFrame()
      dfproducto['Fecha'] = fechas
      dfproducto['Fecha'] = pd.to_datetime(dfproducto['Fecha'])
      dfproducto['Preciomaquina'] = pmaquina
      dfproducto['Precioestanco'] = precio
      dfproducto = dfproducto[dfproducto['Fecha'].dt.year < 2017] # Nuestros datos de ventas acaban en 2016, por lo que eliminamos cambios de precios posteriores a ese año.
      dfproducto.to_excel('drive/MyDrive/Prueba_tecnica/{}_{}.xlsx'.format(marca, producto), index = False)


## Alineacion de precios con los datos de ventas


In [ ]:
# Los precios de los productos cambian con el tiempo, en fechas diferentes. Vamos a obtener el valor correspondiente a cada mes para el que tenemos datos de ventas.
fechas = ventas.Fecha.unique() # Lista con las fechas para las que tenemos ventas.
precios_productos = pd.DataFrame()

for prod in nombres_xlsx: # Iteramos sobre la lista de los nombres de los tabacos que hemos guardado anteriormente.
  dfproducto = pd.read_excel(prod) # Leemos el archivo con el historico de los precios.
  fechas_producto = dfproducto.Fecha.unique()

  precio_por_fecha = []
  for i, val in enumerate(fechas):  # fechas de ventas.
    # Desglosamos las fechas de ventas para poder compararlas con las fechas de las tablas de los precios.
    año = fechas[i].astype('datetime64[Y]').astype(int) + 1970
    mes = fechas[i].astype('datetime64[M]').astype(int) % 12 + 1
    dia = ((fechas[i] - fechas[i].astype('datetime64[M]'))/86400000000000 + 1).astype(int)

    # Desglosamos las fechas de los cambios de precio.
    años_producto = [fechas_producto[e].astype('datetime64[Y]').astype(int) + 1970 for e,val in enumerate(fechas_producto)]
    meses_producto = [fechas_producto[e].astype('datetime64[M]').astype(int) % 12 + 1 for e,val in enumerate(fechas_producto)]
    dias_producto = [((fechas_producto[e] - fechas_producto[e].astype('datetime64[M]'))/86400000000000 + 1).astype(int) for e,val in enumerate(fechas_producto)]

    # Para cada fecha para la que tenemos datos sobre ventas, vamos a seleccionar el precio más cercano en el mes de las ventas.
    closest_dates = [date for date in fechas_producto if date < fechas[i]]
    if len(closest_dates) > 0:
      target_date = max(closest_dates)
      precio_por_fecha.append(float(dfproducto.loc[dfproducto['Fecha'] == target_date, 'Precioestanco']))
    else:
      target_date = min(fechas_producto)
      precio_por_fecha.append(float(dfproducto.loc[dfproducto['Fecha'] == target_date, 'Precioestanco']))

  precios_productos[prod.split('/')[-1].split('.')[0]] = precio_por_fecha

precios_productos.to_excel('drive/MyDrive/Prueba_tecnica/precios.xlsx', index = False) # Guardamos el documento con todos los precios alineados en fecha con los datos de ventas.

In [ ]:
# Juntamos todos los sets de datos ya procesados y guardamos para su futuro uso.
dfs = [ventas, pernoc, parodf, precios_productos]
data = pd.concat(dfs,axis=1)
data.set_index('Fecha', inplace = True)
data.to_excel('drive/MyDrive/Prueba_tecnica/data.xlsx', index = False)

# Analisis Exploratorio

In [ ]:
data = pd.read_excel('drive/MyDrive/Prueba_tecnica/data.xlsx')
ventas = data.iloc[:,:51]
pernoctaciones = data.iloc[:,51:53]
paro = data.iloc[:,53:54]
precios = data.iloc[:,54:]

In [ ]:
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 20

plt.rc('font', size=12)
plt.rc('axes', titlesize=12)
plt.rc('axes', labelsize=12)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('legend', fontsize=12)
plt.rc('figure', titlesize=12)

In [ ]:
# df = ventas.iloc[:,0]
# df['media'] = df.rolling(window=3).mean()
# ventas['Media Móvil'] = df['media']

In [ ]:
# Series temporales de las ventas totales en España.
fig, ax = plt.subplots()
ventas.iloc[:,0].plot(figsize= (10,4), ax = ax, legend = 'Ventas totales')
ventas.iloc[:,-1].plot(color = 'pink', legend = 'Media Móvil')
plt.xlabel('Fecha')
plt.ylabel('Ventas')
plt.xticks(np.arange(0,48,12.0), ['2013', '2014', '2015', '2016'])
plt.title('Ventas Totales')
# plt.legend()
plt.tight_layout()
plt.savefig('drive/MyDrive/Prueba_tecnica/figuras/series_originales_vt', bbox_inches = 'tight')
plt.show()

In [ ]:
# Series temporales de las ventas por provincia.
fig, ax = plt.subplots()
ventas.iloc[:,1:].plot(figsize= (20,8), ax = ax)
plt.xlabel('Fecha')
plt.ylabel('Ventas')
plt.xticks(np.arange(0,48,12.0), ['2013', '2014', '2015', '2016'])
plt.title('Ventas por Provincias')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left', ncol=2)
plt.tight_layout()
plt.savefig('drive/MyDrive/Prueba_tecnica/figuras/series_originales_provincias', bbox_inches = 'tight')
plt.show()

In [ ]:
# Series temporales de los cambios de precios en cada producto para el que tenemos datos.
precios = pd.read_excel('drive/MyDrive/Prueba_tecnica/precios.xlsx')
precios.plot(figsize= (20,8))
plt.xlabel('Fecha')
plt.xticks(np.arange(0,48,12.0), ['2013', '2014', '2015', '2016'])
plt.ylabel('Precios')
plt.title('Cambios en precio de productos')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left', ncol=2)
plt.tight_layout()
plt.savefig('drive/MyDrive/Prueba_tecnica/figuras/precios', bbox_inches = 'tight')
plt.show()

In [ ]:
# Vamos a estudiar que proporcion de las ventas totales es explicada por las ventas de cada provincia, en total y por año.
proporciones = ventas.copy()

In [ ]:
proporciones = (proporciones.iloc[:,1:].div(proporciones.iloc[:,0], axis=0))*100

In [ ]:
# Proporciones medias entre 2013 y 2016
proporciones.mean(axis = 0) # Madrid y BCN, mayores contribuentes con diferencia (>10% de ventas), el tercero estando en un 5%

In [ ]:
# Visualizacion de las proporciones medias para todo el periodo
listprop = proporciones.mean(axis = 0)
p = pd.DataFrame(columns = proporciones.columns)
p.loc[0] = listprop
p = p.T.sort_values(0, ascending=False).T

plt.figure(figsize=(40,1))
plot = sns.heatmap(p,annot = True)
plot.tick_params(left=False, bottom=False)
plt.tight_layout()
plt.savefig('drive/MyDrive/Prueba_tecnica/figuras/proporciones13_16', bbox_inches = 'tight')
plt.show()

In [ ]:
proporciones.std(axis = 0) # Parece que zonas mas turisticas (Balearas, BCN, Girona, MAD, ALicante, Málaga ...) tienen mas variabilidad que el resto.

In [ ]:
# 2013
proporciones.iloc[:12,:].mean(axis = 0)

# Visualizacion de las proporciones medias para 2013
listprop = proporciones.iloc[:12,:].mean(axis = 0)
p = pd.DataFrame(columns = proporciones.columns)
p.loc[0] = listprop
p = p.T.sort_values(0, ascending=False).T

plt.figure(figsize=(40,1))
sns.heatmap(p,annot = True)
plt.tight_layout()
plt.savefig('drive/MyDrive/Prueba_tecnica/figuras/proporciones13', bbox_inches = 'tight')
plt.show()

In [ ]:
# 2014
proporciones.iloc[12:24,:].mean(axis = 0)

# Visualizacion de las proporciones medias para 2014
listprop = proporciones.iloc[12:24,:].mean(axis = 0)
p = pd.DataFrame(columns = proporciones.columns)
p.loc[0] = listprop
p = p.T.sort_values(0, ascending=False).T

plt.figure(figsize=(40,1))
sns.heatmap(p,annot = True)
plt.tight_layout()
plt.savefig('drive/MyDrive/Prueba_tecnica/figuras/proporciones14', bbox_inches = 'tight')
plt.show()

In [ ]:
# 2015
proporciones.iloc[24:36,:].mean(axis = 0)

# Visualizacion de las proporciones medias para 2015
listprop = proporciones.iloc[24:36,:].mean(axis = 0)
p = pd.DataFrame(columns = proporciones.columns)
p.loc[0] = listprop
p = p.T.sort_values(0, ascending=False).T

plt.figure(figsize=(40,1))
sns.heatmap(p,annot = True)
plt.tight_layout()
plt.savefig('drive/MyDrive/Prueba_tecnica/figuras/proporciones15', bbox_inches = 'tight')
plt.show()

In [ ]:
# 2016
proporciones.iloc[36:,:].mean(axis = 0)

# Visualizacion de las proporciones medias para 2016
listprop = proporciones.iloc[36:,:].mean(axis = 0)
p = pd.DataFrame(columns = proporciones.columns)
p.loc[0] = listprop
p = p.T.sort_values(0, ascending=False).T

plt.figure(figsize=(40,1))
sns.heatmap(p,annot = True)
plt.tight_layout()
plt.savefig('drive/MyDrive/Prueba_tecnica/figuras/proporciones16', bbox_inches = 'tight')
plt.show()

In [ ]:
# Identificamos los principales cambios en la aportacion de las ventas por provincia de un año al siguiente.
cambio1314 = dict(zip((proporciones.iloc[:12,:].mean(axis = 0) - proporciones.iloc[12:24,:].mean(axis = 0)).index, (proporciones.iloc[:12,:].mean(axis = 0) - proporciones.iloc[12:24,:].mean(axis = 0)).values))
max_change = max(cambio1314.items(), key=operator.itemgetter(1))[0]
max_val = cambio1314.get(max(cambio1314, key=cambio1314.get))
min_change = min(cambio1314.items(), key=operator.itemgetter(1))[0]
min_val = cambio1314.get(min(cambio1314, key=cambio1314.get))
print('Cambios mas grandes 2013-2014:', max_change, max_val, ', y, ', min_change, min_val)

cambio1415 = dict(zip((proporciones.iloc[12:24,:].mean(axis = 0) - proporciones.iloc[24:36,:].mean(axis = 0)).index, (proporciones.iloc[12:24,:].mean(axis = 0) - proporciones.iloc[24:36,:].mean(axis = 0)).values))
max_change = max(cambio1415.items(), key=operator.itemgetter(1))[0]
max_val = cambio1415.get(max(cambio1415, key=cambio1415.get))
min_change = min(cambio1415.items(), key=operator.itemgetter(1))[0]
min_val = cambio1415.get(min(cambio1415, key=cambio1415.get))
print('Cambios mas grandes 2014-2015:', max_change, max_val, ', y, ', min_change, min_val)

cambio1516 = dict(zip((proporciones.iloc[24:36,:].mean(axis = 0) - proporciones.iloc[36:,:].mean(axis = 0)).index, (proporciones.iloc[24:36,:].mean(axis = 0) - proporciones.iloc[36:,:].mean(axis = 0)).values))
max_change = max(cambio1516.items(), key=operator.itemgetter(1))[0]
max_val = cambio1516.get(max(cambio1516, key=cambio1516.get))
min_change = min(cambio1516.items(), key=operator.itemgetter(1))[0]
min_val = cambio1516.get(min(cambio1516, key=cambio1516.get))
print('Cambios mas grandes 2015-2016:', max_change, max_val, ', y, ', min_change, min_val)

cambio1316 = dict(zip((proporciones.iloc[:12,:].mean(axis = 0) - proporciones.iloc[36:,:].mean(axis = 0)).index, (proporciones.iloc[:12,:].mean(axis = 0) - proporciones.iloc[36:,:].mean(axis = 0)).values))
max_change = max(cambio1316.items(), key=operator.itemgetter(1))[0]
max_val = cambio1316.get(max(cambio1316, key=cambio1316.get))
min_change = min(cambio1316.items(), key=operator.itemgetter(1))[0]
min_val = cambio1316.get(min(cambio1316, key=cambio1316.get))
print('Cambios mas grandes 2013-2016:', max_change, max_val, ', y, ', min_change, min_val)

Cambios mas grandes 2013-2014: Sevilla 0.07473234837942755 , and,  Málaga -0.0838645210876865
Cambios mas grandes 2014-2015: Girona 0.1625267388728293 , and,  Málaga -0.24981942051319406
Cambios mas grandes 2015-2016: Madrid 0.1781310296260301 , and,  Cáceres -0.14276514477567137
Cambios mas grandes 2013-2016: Madrid 0.3682363497762964 , and,  Málaga -0.461291094609924


In [ ]:
correlaciones = pd.read_excel('drive/MyDrive/Prueba_tecnica/metrics/corrmatrix.xlsx')
correlaciones['columnas'] = correlaciones.columns
correlaciones.set_index('columnas', inplace = True)

In [ ]:
# Paro
fig, ax = plt.subplots()
paro.plot(figsize= (5,2), ax = ax, color = 'red')
plt.xlabel('Fecha')
plt.ylabel('Paro')
plt.xticks(np.arange(0,48,12.0), ['2013', '2014', '2015', '2016'])
plt.title('Paro')
plt.tight_layout()
plt.show()

In [ ]:
# Ventas vs paro
plt.figure(figsize = (5,2))
plt.scatter(data.iloc[:,0], paro, color = 'red')
plt.xlabel('Ventas Totales')
plt.ylabel('Paro')
plt.title('Paro vs Ventas. Correlacion: {}'.format(-0.14))
plt.tight_layout()
plt.show()

In [ ]:
# Pernoctaciones
fig, ax = plt.subplots()
pernoctaciones.iloc[:,1].plot(figsize= (5,2), ax = ax, color = 'green')
plt.xlabel('Fecha')
plt.ylabel('Pernoctaciones')
plt.xticks(np.arange(0,48,12.0), ['2013', '2014', '2015', '2016'])
plt.title('Pernoctaciones')
plt.tight_layout()
plt.show()

In [ ]:
# Ventas vs pernoctaciones.
plt.figure(figsize = (5,2))
plt.scatter(data.iloc[:,0], data['Pernoctaciones (INE)'], color = 'green')
plt.xlabel('Ventas Totales')
plt.ylabel('Pernoctaciones')
plt.title('Pernoctaciones vs Ventas. Correlacion: {}'.format(0.80))
plt.tight_layout()
plt.show()

# Analisis Estadístico

In [ ]:
from functools import reduce
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.vector_ar.vecm import coint_johansen

In [ ]:
data = pd.read_excel('drive/MyDrive/Prueba_tecnica/data.xlsx')

In [ ]:
# Definimos tests estadisticos para comprobar si las series temporales son estacionarias. Este paso es imprescindible antes de modelar.
def adf_test(timeseries): # Dickey-Fuller
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
       dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)
    return dfoutput

def kpss_test(timeseries): # Kwiatkowski-Phillips-Schmidt-Shin
  kpsstest = kpss(timeseries, regression='c')
  kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic','p-value','Lags Used'])
  for key,value in kpsstest[3].items():
    kpss_output['Critical Value (%s)'%key] = value
  return kpss_output

In [ ]:
estacionarias = []
non = []
for var in data.columns:
  dfoutput = kpss_test(data[var])
  if dfoutput['p-value'] <= 0.1:
    estacionarias.append(var)
  else:
    non.append(var)

# Todas las series son estacionarias.
if len(estacionarias) == len(data.columns):
  print('Todas las series son estacionarias.')

## Similitud entre series temporales.

- Se van a usar dos métodos para determinar la similitud entre las series temporales, y así decidir cuáles son mejores predictores de cada una: Dynamic Time Warping y Coeficiente de correlacion de Spearman.

In [ ]:
from dtw import dtw,accelerated_dtw
from sklearn.preprocessing import MinMaxScaler

# Para calcular DTW, se normalizan las series, asi comprobamos si las series son similares en "forma", sin importar diferencias en magnitud.
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(data)
datanorm = pd.DataFrame(scaled, columns = data.columns)

# Con estos dos loops se genera la matriz de distancias de DTW para cada par de series.
listaglobal = []
for var1 in data.columns:
  listvar1 = []
  for var2 in data.columns:
    d, _, _, path = accelerated_dtw(datanorm[var1].values,datanorm[var2].values, dist='euclidean')
    listvar1.append(d)
  listaglobal.append(listvar1)

dtwmatrix = pd.DataFrame(listaglobal, columns = data.columns)
dtwmatrix.set_index(data.columns, inplace = True)
dtwmatrix.to_excel('drive/MyDrive/Prueba_tecnica/metrics/dtwmatrix.xlsx', index=False)
dtwmatrix = -dtwmatrix # Ya que más tarde utilizaremos la correlacion, y valores más grandes en magnitud representan mayor similitud, aquí cogemos el valor negativo de las distancias, para que sea igual.
top5dtw = np.array([dtwmatrix[c].nlargest(6).index.values for c in dtwmatrix]) # Para cada serie, guardamos las otras 5 series más similares a ella. El modelo que usamos para predecir tiene ciertos constraints, y
                                                                               # por ello, solo podemos usar 5 series para predecir otra.

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(-dtwmatrix,annot = False)
plt.show()

In [ ]:
# Calculamos la correlacion de spearman entre pares de series
corr = data.corr(method = 'spearman')
corr.to_excel('drive/MyDrive/Prueba_tecnica/metrics/corrmatrix.xlsx', index=False)
top5corr = np.array([np.abs(corr)[c].nlargest(6).index.values for c in np.abs(corr)]) # Guardamos las 5 series mas correlacionadas con cada serie.

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(np.abs(corr),annot = False)
plt.show()

# Vector Auto Regression model (VAR)

Para realizar la prediccion de los valores futuros de ventas de cada serie, vamos a utilizar VAR, que nos permite predecir valores de multiples series simultaneamente, bajo las condiciones de que las series sean estacionarias (comprobado en el apartado de "Analisis estadistico") y que se afecten entre ellas (comprobado con el estudio de similaridad).


In [ ]:
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.stattools import acf

In [ ]:
# Función que usaremos mas tarde para estudiar el performance de los modelos en la prediccion de valores futuros de cada serie.
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    mae = np.mean(np.abs(forecast - actual))    # MAE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    return [mape,mae,rmse]

In [ ]:
# Toda esta celda se repite usando los 5 mejores valores de correlacion y de dtw. Se guardan los .xlsx de las predicciones y de las metricas obtenidas para luego comprobar qué metodo da mejores resultados para cada serie.
df_forecast_corr = pd.DataFrame()
accdf_corr = pd.DataFrame()
topsel = 'corr'

for idx, val in enumerate(top5corr[:51]): # top5corr es la lista con las 5 series mas similares a la serie target, para cada serie target. 51 son el numero de series que queremos predecir (ventas totales y porivncias).
  train = data[top5corr[idx]].iloc[:-6] # entrenamos solo con las 5 series seleccionadas, y dejamos 6 de las 48 observaciones para testear el performance.
  test = data[top5corr[idx]].iloc[-6:]

  model = VAR(train)
  model2 = model.select_order(maxlags=5)
  model_fitted = model.fit(5) # El modelo se va entrenar utilizando los 5 valores anteriores. Debido a la limitación en el numero de observaciones, el modelo de VAR no puede realizar estimaciones utilizando un numero mayor de muestras.

  lag = model_fitted.k_ar
  forecast_input = train.values[-lag:]

  nobs = 6
  fc = model_fitted.forecast(y=forecast_input, steps=nobs) # Generamos 6 observaciones con el modelo entrenado, para comparar con las 6 observaciones de test.

  df_forecast_corr[top5corr[idx][0]] = fc[:,0]
  df_forecast_corr.to_excel('drive/MyDrive/Prueba_tecnica/metrics/forecast_corr.xlsx',index = False) # Guardamos los valores predichos para cada serie.

  column = top5corr[idx][0]
  print('COLUMNA', column)
  accdf_corr[column] = forecast_accuracy(df_forecast_corr[column].values, test[column].values)
  accdf_corr['metric'] = ['mape','mae','rmse']
  accdf_corr.set_index('metric', inplace = True)
  accdf_corr.to_excel('drive/MyDrive/Prueba_tecnica/metrics/metrics_forecast_corr.xlsx',index = False) # Guardamos las metricas obtenidas de comparar valores reales y predichos.

  # Aqui mostramos los graficos de valores predichos vs valores
  plt.figure(figsize=(12,5))
  plt.xlabel('Prediccion vs Real: {}'.format(column))
  ax1 = df_forecast_corr[column].plot(color='blue', grid=True, label='FC',use_index = False)
  ax2 = test[column].plot(color='red', grid=True, secondary_y=True, label='Test',use_index = False)
  h1, l1 = ax1.get_legend_handles_labels()
  h2, l2 = ax2.get_legend_handles_labels()
  plt.legend(h1+h2, l1+l2, loc=2)
  plt.tight_layout()
  #plt.show()
  plt.savefig('drive/MyDrive/Prueba_tecnica/figuras/forecasted_{}_{}'.format(column.split('/')[0], topsel), bbox_inches = 'tight')

In [ ]:
# La celda anterior se ha repetido utilizando DTW y Correlacion para seleccionar los 5 mejores predictores de cada serie. Comparando las metricas obtenidas, podemos identificar qué metodo es mejor en cada caso, ç
# Para luego utilizarlo a la hora de predecir los valores de 2017.
bestpercol = []
for var in accdf_corr.columns:
  corrdata = accdf_corr[var]
  dtwdata = accdf_dtw[var]
  subs = corrdata - dtwdata # Si la resta de las metricas es positiva, el valor con DTW es menor, indicando que es mejor opcion que la correlacion para elegir predictores.
  totalbetterdtw = sum(subs>0)
  if totalbetterdtw >= 2: # Se estudian 3 metricas: MAPE, MAE, y RMSE, si 2 o mas de ellas son superiores usando DTW, se usara DTW para la prediccion final, sino, se usara la correlacion.
    bestpercol.append('dtw')
  else:
    bestpercol.append('corr')

In [ ]:
# Las ventas totales son la suma de ventas marginales de cada provincia. A pesar de tener una prediccion especifica para las ventas totales, vamos a comprobar si se obtienen mejores resultados
# sumando las predicciones de cada provincia
df_forecast_corr['VT Suma']  = df_forecast_corr.iloc[:,1:].sum(axis = 1)
df_forecast_dtw['VT Suma']  = df_forecast_dtw.iloc[:,1:].sum(axis = 1)
# Los errores son mas grandes con las sumas (en ambos casos), asique mejor predecir las ventas totales con su propio modelo.
accdf_corr.iloc[:,0].drop(['me','mpe','corr','minmax']) - accdf_corr['VT Suma'].drop(['me','mpe','corr','minmax'])
accdf_dtw.iloc[:,0].drop(['me','mpe','corr','minmax']) - accdf_dtw['VT Suma'].drop(['me','mpe','corr','minmax'])

In [ ]:
acccorr = pd.read_excel('drive/MyDrive/Prueba_tecnica/metrics/metrics_forecast_corr.xlsx').drop([1,3,5,6])
acccorr['metric'] = ['MAPE', 'MAE', 'RMSE']
acccorr.set_index('metric', inplace = True)

accdtw = pd.read_excel('drive/MyDrive/Prueba_tecnica/metrics/metrics_forecast_dtw.xlsx').drop([1,3,5,6])
accdtw['metric'] = ['MAPE', 'MAE', 'RMSE']
accdtw.set_index('metric', inplace = True)

# Para poder estudiar el modelo final, tenemos que guardar las metricas para cada provincia con su metodo (DTW/correlacion)
finalperfromance = pd.DataFrame()
finalperfromance['metric'] = ['MAPE', 'MAE', 'RMSE']
finalperfromance.set_index('metric', inplace = True)

for idx,var in enumerate(acccorr.columns.to_list()):
  if bestpercol[idx]=='corr':
    df = acccorr
  else:
    df = accdtw
  finalperfromance[var] = df[var]

finalperfromance.to_excel('drive/MyDrive/Prueba_tecnica/metrics/final_acc.xlsx', index = False)

In [ ]:
# Habiendo comprobado el performance de los modelos para cada provincia, y habiendo determinado que metodo es mejor para seleccionar predictores, vamos a predecir los valores para 2017 para cada provincia independientemente;
# El codigo es prácticamente identico al anterior.
df_forecast = pd.DataFrame()

for idx, val in enumerate(top5corr[:51]):

  if bestpercol[idx] == 'corr':
    top5 = top5corr[idx]
  else:
    top5 = top5dtw[idx]

  train = data[top5]

  model = VAR(train)
  model_fitted = model.fit(5)

  lag = model_fitted.k_ar
  forecast_input = train.values[-lag:]
  nobs = 12
  fc = model_fitted.forecast(y=forecast_input, steps=nobs) # predecimos 12 en vez de 6 porque queremos predecir todo 2017.
  df_forecast[top5[0]] = fc[:,0]

df_forecast.to_excel('drive/MyDrive/Prueba_tecnica/metrics/forecast.xlsx',index = False)

In [ ]:
# Se juntan los datos originales y las predicciones para 2017.
data_plus_predictions = data.iloc[:,:df_forecast.shape[1]].append(df_forecast, ignore_index = True)
dforig = pd.read_excel('drive/MyDrive/Prueba_tecnica/Datos entrevista.xlsx', skiprows=2)
data_plus_predictions['Fecha'] = dforig['Fecha']
data_plus_predictions.set_index('Fecha', inplace = True)
data_plus_predictions.to_excel('drive/MyDrive/Prueba_tecnica/metrics/final_data.xlsx', index = False)
data_plus_predictions.to_excel('drive/MyDrive/Prueba_tecnica/metrics/final_data.xlsx', index = False)

In [ ]:
data_plus_predictions = pd.read_excel('drive/MyDrive/Prueba_tecnica/metrics/final_data.xlsx')

In [ ]:
# Series temporales de ventas totales incluyendo valores predichos.
fig, ax = plt.subplots()
data_plus_predictions.iloc[:,0].plot(figsize= (16,5), ax = ax)
plt.axvspan(48, 60, color='lightgray', alpha=0.5, lw=0)
plt.xlabel('Fecha')
plt.ylabel('Ventas')
plt.xticks(np.arange(0,60,12.0), ['2013', '2014', '2015', '2016', '2017'])
plt.title('Ventas Totales + Predicciones para 2017')
plt.legend()
plt.tight_layout()
plt.savefig('drive/MyDrive/Prueba_tecnica/figuras/series_mas_predicciones_totales', bbox_inches = 'tight')
plt.show()

In [ ]:
# Series temporales de ventas por provincias incluyendo valores predichos.
fig, ax = plt.subplots()
data_plus_predictions.iloc[:,1:].plot(figsize= (20,8), ax = ax)
plt.axvspan(48, 60, color='lightgray', alpha=0.5, lw=0)
plt.xlabel('Fecha')
plt.ylabel('Ventas')
plt.xticks(np.arange(0,60,12.0), ['2013', '2014', '2015', '2016', '2017'])
plt.title('Ventas por Provincia + Predicciones para 2017')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left', ncol=2)
plt.tight_layout()
plt.savefig('drive/MyDrive/Prueba_tecnica/figuras/series_mas_predicciones_provincias', bbox_inches = 'tight')
plt.show()

# Cambios en los precios.



In [ ]:
correlaciones = pd.read_excel('drive/MyDrive/Prueba_tecnica/metrics/corrmatrix.xlsx')
correlaciones['columnas'] = correlaciones.columns
correlaciones.set_index('columnas', inplace = True)

In [ ]:
correlaciones.rename({'Ventas total España (sin Canarias)': 'Ventas Totales'}, inplace = True)

In [ ]:
# Como vemos, las correlaciones son ligeras +-0.3, y sorprendentemente en su mayoria positivas, indicando que la subida de precios no ha frenado las ventas historicamente. Cadiz y Málaga son las "menos afectadas"
# aparentemente.
# Aparecen aun asi casos de correlaciones negativas: Badajoz, Madrid, Guipuzcoa, Jaen y LLeida destacan en este aspecto. Subidas de precio disminuyen ventas (pero la relacion es baja). Siendo
# Madrid una de las provincias donde mas se vende, quizás podría tener cierto impacto una subida de precios.
plt.figure(figsize=(20,10))
sns.heatmap(correlaciones.iloc[:51,54:].T,annot = False)
plt.tight_layout()
plt.savefig('drive/MyDrive/Prueba_tecnica/figuras/correlacion_ventas_precios', bbox_inches = 'tight')
plt.show()

In [ ]:
# Ya se vio en un apartado anterior que la serie de las ventas totales es estacionaria: la media y varianza son constantes en el tiempo. Teniendo en cuenta que el precio ha ido subiendo, se podría decir de base
# que las subidas de precio no han tenido impactos en las ventas. Hay que comprobar si la subida de precios ha sido >= 10%.
data = pd.read_excel('drive/MyDrive/Prueba_tecnica/data.xlsx')
precios = data.iloc[:,54:]

In [ ]:
cambios_precios = [(max(precios[i].values) - min(precios[i].values))/min(precios[i].values) for i in precios.columns]
print('En el periodo comprendido entre 2013 y 2016, los precios de las cajetillas de tabaco han cambiado en un {} %'.format(np.mean(cambios_precios)*100))

In [ ]:
correlaciones.iloc[0,54:].max()